### Install packages

In [ ]:
%%bash
pip install -qqq torch
pip install -qqq transformers
pip install -qqq sentence-transformers
pip install -qqq datasets
pip install -qqq faiss-cpu
pip install -qqq bitsandbytes
pip install -qqq accelerate
pip install -qqq nvidia-ml-py3
pip install -qqq langchain

### Imports

In [1]:
import psutil
import warnings
import nvidia_smi

import torch
import gzip
import shutil

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from time  import time

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import  LLMChain
from langchain_community.document_loaders.csv_loader import CSVLoader

/home/andrea/anaconda3/envs/hmd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Utils

In [2]:
def show_usage():
  print(f"Current CPU usage: {psutil.cpu_percent(3)}%")
  print(f"Current RAM usage: {psutil.virtual_memory()[3]/pow(1024,3):.2f} GB/{psutil.virtual_memory()[0]/pow(1024,3):.2f} GB")

  nvidia_smi.nvmlInit()

  deviceCount = nvidia_smi.nvmlDeviceGetCount()
  for i in range(deviceCount):
      handle = nvidia_smi.nvmlDeviceGetHandleByIndex(i)
      info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
      print("{}, Memory : ({:.2f}% free): {:.2f}GB (total), {:.2f}GB (free), {:.2f}GB (used)".format(nvidia_smi.nvmlDeviceGetName(handle), 100*info.free/info.total, info.total/pow(1024,3), info.free/pow(1024,3), info.used/pow(1024,3)))

  nvidia_smi.nvmlShutdown()

def unzip_gzip(file_path):
  with gzip.open(file_path, 'rb') as f_in:
      with open(file_path[:-3], 'wb') as f_out:
          shutil.copyfileobj(f_in, f_out)
  return file_path[:-3]

warnings.filterwarnings('ignore')

show_usage()


Current CPU usage: 0.1%
Current RAM usage: 1.71 GB/15.50 GB
b'NVIDIA GeForce RTX 3070 Ti', Memory : (87.73% free): 8.00GB (total), 7.02GB (free), 0.98GB (used)


## FAISS-Mistral

### Mistral

In [4]:
model_name="mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True
)

show_usage()

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    task="text-generation",
    top_k=5,
    repetition_penalty=1.1,
    num_return_sequences=1,
    return_full_text=True,
    max_new_tokens=300,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

prompt_template = """<s>[INST] You are a helpful, respectful and honest assistant. Answer exactly in few words from the context
Answer the question below from context below :
{context}

{question} [/INST] </s>
"""

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.18s/it]


Current CPU usage: 0.2%
Current RAM usage: 2.67 GB/15.50 GB
b'NVIDIA GeForce RTX 3070 Ti', Memory : (2.63% free): 8.00GB (total), 0.21GB (free), 7.79GB (used)


### FAISS

In [3]:
data_path = "./data/KB_user_profiles.csv.gz"
data_path = unzip_gzip(data_path)

loader = CSVLoader(data_path)
data = loader.load()

In [6]:
print(data[1].metadata)

{'source': './data/KB_user_profiles.csv', 'row': 1}


In [8]:
# db = FAISS.from_documents(data, HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))
# db.save_local("faiss_index")
db = FAISS.load_local("./faiss_index", HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

In [47]:
# FAISS as_retriever NO LLM

query = "I'm looking for a senior sw developer, at least 5 years of experience and a PhD"

retriever = db.as_retriever(search_kwargs={"k":10})
results = retriever.invoke(query)

for result in results:
    print(result.page_content, "\n")

name: Xavier
surname: Marshall
gender: 0
mansion: Research Engineer
category: science
category_id: 4
description: I am Xavier Marshall, a Research Engineer resourceful project manager. With a tenacious fervor for web development, I bring 12 years of experience to the telecommunications domain. Recognized for my proactive project coordination, I excel in leading software development projects and consistently deliver positive outcomes.
salary: 5070
skills: Systems Approach, Intercultural Communication, Cultural Competence, Initiative, Team Building
city: Tallinn
country: Estonia
remote_hybrid_office: 2
relocation: 1
current_company: BlueSkyEnergy Solutions
part_full_time: 1
junior_senior: 0
experience_years: 15
level_education: 4 

name: Felix
surname: Armstrong
gender: 0
mansion: Back-End Developer
category: other
category_id: 8
description: I am Felix Armstrong, a Back-End Developer proactive project manager. With a versatile fervor for web development, I bring 22 years of experience t

In [48]:
docs_and_scores = db.similarity_search_with_score(query)
results = docs_and_scores

for doc, score in results:
    print(doc.page_content, f"\nscore: {score}", "\n")

name: Xavier
surname: Marshall
gender: 0
mansion: Research Engineer
category: science
category_id: 4
description: I am Xavier Marshall, a Research Engineer resourceful project manager. With a tenacious fervor for web development, I bring 12 years of experience to the telecommunications domain. Recognized for my proactive project coordination, I excel in leading software development projects and consistently deliver positive outcomes.
salary: 5070
skills: Systems Approach, Intercultural Communication, Cultural Competence, Initiative, Team Building
city: Tallinn
country: Estonia
remote_hybrid_office: 2
relocation: 1
current_company: BlueSkyEnergy Solutions
part_full_time: 1
junior_senior: 0
experience_years: 15
level_education: 4 
score: 0.8464797735214233 

name: Felix
surname: Armstrong
gender: 0
mansion: Back-End Developer
category: other
category_id: 8
description: I am Felix Armstrong, a Back-End Developer proactive project manager. With a versatile fervor for web development, I bri

In [6]:
retriever = db.as_retriever(search_kwargs={"k":3})
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

In [7]:
t0 = time()
output = rag_chain.invoke("Find me the best candidate for a software engineering position")
print(f"{time()-t0:.2f} seconds for generation")

show_usage()

105.34 seconds for generation
Current CPU usage: 0.1%
Current RAM usage: 4.86 GB/15.50 GB
b'NVIDIA GeForce RTX 3070 Ti', Memory : (2.11% free): 8.00GB (total), 0.17GB (free), 7.83GB (used)


In [9]:
print(output["text"])

Based on the provided context, all three candidates (Calypso Matthews, Isabel Long, and Elowen Edwards) have experience in leading software development projects. However, the best candidate for a software engineering position would depend on specific requirements and qualifications needed for the role. Here's a brief comparison of their skills mentioned in the context:

* Calypso Matthews: Facilitation Skills, Critical Analysis, Learning Agility, Networking, Analytical Thinking
* Isabel Long: Change Adaptability, Effective Communication, Intrapreneurship, Curiosity, Experimentation
* Elowen Edwards: Data-Driven Decision-Making, Conflict Resolution, Influence and Persuasion, Interdisciplinary Collaboration, Cognitive Diversity

Without knowing which skills are most important for the software engineering position, it is difficult to determine which candidate is the best fit based on the given context alone. It may be necessary to consider other factors such as education level, experience